In [1]:
import pandas as pd
import numpy as np

In [29]:
def conf_int_month(data, name):
    prognosis_month = pd.DataFrame(data.groupby(
        pd.Grouper(freq='M')).mean())

    prognosis_month = prognosis_month.iloc[-4:, :]

    prognosis_month['std'] = pd.DataFrame(data.groupby(
        pd.Grouper(freq='M')).std())

    prognosis_month['num_days'] = pd.DataFrame(data.groupby(
        pd.Grouper(freq='M')).count())

    prognosis_month[f'{name}_min'] = prognosis_month[f'{name}'] - 1.96 * \
        (prognosis_month['std'] / np.sqrt(prognosis_month['num_days']))

    prognosis_month[f'{name}_max'] = prognosis_month[f'{name}'] + 1.96 * \
        (prognosis_month['std'] / np.sqrt(prognosis_month['num_days']))

    prognosis_month = prognosis_month.drop(
        ['std', 'num_days'], axis=1)

    return prognosis_month


def conf_int_quarter(data, name):
    prognosis__q = pd.DataFrame(
        data.groupby(pd.Grouper(freq='Q')).mean())

    prognosis__q['std'] = pd.DataFrame(
        data.groupby(pd.Grouper(freq='Q')).std())

    prognosis__q['num_days'] = pd.DataFrame(
        data.groupby(pd.Grouper(freq='Q')).count())

    prognosis__q[f'{name}_min'] = prognosis__q[f'{name}'] - 1.96 * \
        (prognosis__q['std'] / np.sqrt(prognosis__q['num_days']))

    prognosis__q[f'{name}_max'] = prognosis__q[f'{name}'] + 1.96 * \
        (prognosis__q['std'] / np.sqrt(prognosis__q['num_days']))

    prognosis__q = prognosis__q.drop(columns=['std', 'num_days'])

    prognosis__q = prognosis__q.iloc[-1:, :]

    prognosis__q = prognosis__q.rename(
        index={prognosis__q.index[0]: 'Quarter'})

    return prognosis__q

In [32]:
data_ar = pd.read_csv('./results/intermediate/spot_ar_prognosis_copper.csv', parse_dates=['index'])
data_ar = pd.DataFrame(data_ar.iloc[:,:2])
data_ar = data_ar.rename(columns={data_ar.columns.to_list()[1]:'AutoReg', 'index':'date'})
data_ar['dow'] = data_ar['date'].dt.day_of_week
data_ar = data_ar[(data_ar['dow'] != 5) & (data_ar['dow'] != 6)]
data_ar.drop(['dow'], axis=1, inplace=True)
data_ar.set_index('date', inplace=True)

ar_month_data = conf_int_month(data_ar, 'AutoReg')
ar_quarter_data = conf_int_quarter(data_ar, 'AutoReg')

result_ar = pd.concat([ar_month_data, ar_quarter_data], axis=0)
result_ar = result_ar.round()

result_ar

,AutoReg,AutoReg_min,AutoReg_max
date,,,
2023-09-30 00:00:00,8265.0,8205.0,8325.0
2023-10-31 00:00:00,8085.0,8079.0,8092.0
2023-11-30 00:00:00,8037.0,8031.0,8043.0
2023-12-31 00:00:00,7994.0,7989.0,8000.0
Quarter,8041.0,8031.0,8051.0


In [34]:
data_dt = pd.read_csv('./results/intermediate/spot_dt_prognosis_copper.csv', parse_dates=['index'])
data_dt = pd.DataFrame(data_dt.iloc[:,:2])
data_dt = data_dt.rename(columns={data_dt.columns.to_list()[1]:'Decision_tree', 'index':'date'})
data_dt['dow'] = data_dt['date'].dt.day_of_week
data_dt = data_dt[(data_dt['dow'] != 5) & (data_dt['dow'] != 6)]
data_dt.drop(['dow'], axis=1, inplace=True)
data_dt.set_index('date', inplace=True)

dt_month_data = conf_int_month(data_dt, 'Decision_tree')
dt_quarter_data = conf_int_quarter(data_dt, 'Decision_tree')

result_dt = pd.concat([dt_month_data, dt_quarter_data], axis=0)
result_dt = result_dt.round()

result_dt

,Decision_tree,Decision_tree_min,Decision_tree_max
date,,,
2023-09-30 00:00:00,8293.0,8223.0,8363.0
2023-10-31 00:00:00,8746.0,8715.0,8778.0
2023-11-30 00:00:00,8412.0,8316.0,8508.0
2023-12-31 00:00:00,8335.0,8230.0,8441.0
Quarter,8506.0,8441.0,8570.0


In [35]:
adj_df = pd.concat([data_ar, data_dt], axis=1)

adj_df['Final'] = np.mean([adj_df['AutoReg'], adj_df['Decision_tree']], axis=0)
adj_df.dropna(inplace=True)

adj_df = pd.DataFrame(adj_df.iloc[:, -1])

adj_month = conf_int_month(adj_df, 'Final')
adj_quarter = conf_int_quarter(adj_df, 'Final')

result_adj = pd.concat([adj_month, adj_quarter], axis=0)
result_adj = result_adj.round()

model_result = pd.concat([result_ar, result_dt, result_adj], axis=1)
model_result

,AutoReg,AutoReg_min,AutoReg_max,Decision_tree,Decision_tree_min,Decision_tree_max,Final,Final_min,Final_max
date,,,,,,,,,
2023-09-30 00:00:00,8265.0,8205.0,8325.0,8293.0,8223.0,8363.0,8279.0,8220.0,8339.0
2023-10-31 00:00:00,8085.0,8079.0,8092.0,8746.0,8715.0,8778.0,8416.0,8402.0,8430.0
2023-11-30 00:00:00,8037.0,8031.0,8043.0,8412.0,8316.0,8508.0,8225.0,8175.0,8275.0
2023-12-31 00:00:00,7994.0,7989.0,8000.0,8335.0,8230.0,8441.0,8165.0,8113.0,8217.0
Quarter,8041.0,8031.0,8051.0,8506.0,8441.0,8570.0,8274.0,8238.0,8309.0
